### The Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/stars.csv')

In [3]:
df.sample(5)

,description,language,repo,stargazers,user
15082,QuIP provides an interactive environment for c...,C,nasa/QuIP,108,periode
11689,Declarative statistical visualization library ...,Python,altair-viz/altair,4208,wanghaolin
17517,NaN,JavaScript,superscriptjs/normalizer,100,dancrew32
22086,Python dependency management and packaging mad...,Python,sdispater/poetry,6329,pavlokurochka
20898,Generate slow-motion videos by interpolating m...,Python,MortenHannemose/pytorch-vfi-cft,90,blanchefort


### The Cleanup

In [4]:
df = df[df['repo'] != 'maxhumber/gazpacho']
df = df[df.language.isin(['Python', 'Jupyter Notebook'])]
popular = pd.DataFrame(df['repo'].value_counts())
select_repos = popular[popular['repo'] >= 5].index.tolist()
df = df[df['repo'].isin(select_repos)]

In [5]:
df.sample(5)

,description,language,repo,stargazers,user
19970,Never use print for debugging again,Python,cool-RR/PySnooper,12726,viseshrp
13201,Turn Python scripts into handouts with Markdow...,Python,danijar/handout,1772,erdimollahuseyin
23810,Async Python 3.6+ web server/framework | Build...,Python,huge-success/sanic,12899,avinash-mishra
18481,The Python micro framework for building web ap...,Python,pallets/flask,46930,berlotto
6428,Comprehensive Python Cheatsheet,Python,gto76/python-cheatsheet,9183,pushpendrapratap


In [6]:
# shove into something that looks like this?
pd.DataFrame([
    [0, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 0, 0, 1]
])

,0,1,2,3
0,0,0,1,0
1,0,1,1,0
2,1,0,0,1
3,0,0,0,1


### The Preparation

In [7]:
df = df.groupby(['user'])['repo'].apply(lambda x: ','.join(x))
df = pd.DataFrame(df)

In [8]:
df.sample(5)

,repo
user,
erdimollahuseyin,"psf/black,idealo/imagededup,streamlit/streamli..."
mxuribe,"sebst/pythonic-news,facebookresearch/hydra,koo..."
stribny,"laike9m/pdir2,eon01/kubernetes-workshop,takluy..."
bergran,"faif/python-patterns,videoflow/videoflow,slori..."
pixelandpen,"streamlit/streamlit,jankrepl/pychubby,hzlmn/di..."


In [9]:
df.loc['Curlybear'].values.tolist()

['vinta/awesome-python,CorentinJ/Real-Time-Voice-Cloning,danijar/handout,ageitgey/face_recognition,jofpin/trape,grapheneX/grapheneX,gto76/python-cheatsheet,trekhleb/homemade-machine-learning,manrajgrover/halo,chris104957/maildown,psf/black,stewartmcgown/uds,TheAlgorithms/Python,cool-RR/PySnooper,Bogdanp/molten,nicolargo/glances,mkaz/termgraph,donnemartin/system-design-primer,chubin/cheat.sh,psf/requests-html,Zulko/moviepy,jarun/Buku,pypa/pipenv,soimort/you-get,public-apis/public-apis,HelloZeroNet/ZeroNet']

### The Model

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


class NNRecommender:
    def __init__(self, n_neighbors=10, max_features=1000, tokenizer=lambda x: x.split(",")):
        self.cv = CountVectorizer(tokenizer=tokenizer, max_features=max_features)
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            neighbors = self.nn.kneighbors(Xt, return_distance=False)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(",")
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(",")]
            Xp.append(repos)
        return Xp

In [11]:
model = NNRecommender()
model.fit(df['repo'])

In [12]:
df['repo'][42]

'CorentinJ/Real-Time-Voice-Cloning,s0md3v/goop,psf/requests,slundberg/shap,facebookresearch/pytext,practicalAI/practicalAI,Zulko/moviepy,python/mypy,ParthS007/background,keon/algorithms,google/python-fire,eriklindernoren/ML-From-Scratch,minimaxir/big-list-of-naughty-strings,pudo/dataset,nvbn/thefuck,dbcli/pgcli,faif/python-patterns,chriskiehl/Gooey'

In [13]:
model.predict([df['repo'][42]])

[['weskerfoot/DeleteFB',
  'deepfakes/faceswap',
  'fastai/fastai',
  'sherlock-project/sherlock',
  'Miserlou/Zappa',
  'shengqiangzhang/examples-of-web-crawlers',
  'smacke/subsync',
  'sloria/TextBlob',
  'scikit-learn/scikit-learn',
  'donnemartin/interactive-coding-challenges',
  'dae/anki',
  'ageron/handson-ml2']]

### Under the Hood

In [14]:
df.iloc[0].values.tolist()

['python/cpython,hugapi/hug,HypothesisWorks/hypothesis,timothycrosley/portray,timothycrosley/hypothesis-auto,tiangolo/fastapi,bocadilloproject/bocadillo,tartiflette/tartiflette,encode/httpx,pytest-dev/pytest,tonybaloney/wily,sdispater/poetry,mahmoud/glom,psf/black,chubin/wttr.in']

In [15]:
cv = CountVectorizer(tokenizer=lambda x: x.split(','), max_features=1000)
X = cv.fit_transform(df['repo'])
X.todense()[:5]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
nn = NearestNeighbors(n_neighbors=10)

In [17]:
nn.fit(X)
ind = nn.kneighbors(return_distance=False)
ind[:5]

array([[106,  14,  99,  48,  65,   3,  10,  19,   2,  40],
       [ 14,  66,  99,  10,  19,  65,  48,   3,  40,  12],
       [ 10,  19,   3,  99,  48,  14,  65,  40,  66,  92],
       [ 48,  10,  99,  19,  14,  65,   2,  40,  92,  72],
       [ 28,  10,  48,  72,  20,  19,  99,  14,  65,   3]])

In [18]:
df.iloc[106].values

array(['timothycrosley/hypothesis-auto,timothycrosley/portray,sdispater/poetry,wyounas/homer,tiangolo/fastapi,psf/black,marshmallow-code/marshmallow,falconry/falcon,cookiecutter/cookiecutter'],
      dtype=object)

#### Pickle 

In [19]:
import pickle

In [20]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

PicklingError: Can't pickle <function NNRecommender.<lambda> at 0x11f5eab70>: attribute lookup NNRecommender.<lambda> on __main__ failed

In [21]:
import dill # pip install dill

with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

del model

with open('model.pkl', 'rb') as f:
    model = dill.load(f)

In [22]:
model.predict(['streamlit/streamlit,huggingface/transformers,encode/httpx,aws/chalice,maxhumber/chart'])

[['sloria/TextBlob',
  'plasticityai/supersqlite',
  'scikit-learn/scikit-learn',
  'shengqiangzhang/examples-of-web-crawlers',
  'Avik-Jain/100-Days-Of-ML-Code',
  'minimaxir/big-list-of-naughty-strings',
  'ytdl-org/youtube-dl',
  'vinta/awesome-python',
  'fastai/fastai',
  'donnemartin/system-design-primer',
  'mherrmann/fbs',
  'TheAlgorithms/Python',
  'PySimpleGUI/PySimpleGUI']]